In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib
import matplotlib.pyplot as plt
print("matplotlib version: {}". format(matplotlib.__version__))

import sklearn # machine learning algorithms
print("scikit-learn version: {}". format(sklearn.__version__))
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

import xgboost as xgb
print("xgb version: {}". format(xgb.__version__))
from xgboost import plot_importance
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import GridSearchCV 

import imblearn # for using oversampling
print("imblearn version: {}". format(imblearn.__version__))
from imblearn.over_sampling import SMOTE

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# About

This is my notebook for the Tabular Playground Series - May 2021 using XGBoost. I have a [seperate notebook for my EDA](https://www.kaggle.com/melanie7744/tps5-eda-raising-more-questions-than-answers).

I frist ran plain XGBoost with default parameters to see how it fares. Then I 
- removed the duplicates from the training set
- one hot encoded four parameters that I guessed to be categorical
- ran Grid Search CV to search for better hyperparameters
- increased the number of estimators and early stopping rounds
- implemented oversampling (SMOTE) to deal with the imbalance in the target variable (corrected later on, initial implementation was incorrect)
- reduced the learning rate

The results are listed as comments below. It was an iterative process with the current version of the notebook only showing the last step.


I ran out of things to try with XGBoost. My biggest issue is, that I did not find a way to deal with the imbalance of the target variable. Stratification during generation of the validation set was not enough, neither was oversampling. As a last tuning step reducing the learning rate helped to get a better log loss score. But only slightly. 

Any suggesetions are very welcome!

In [ ]:
df_train = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
df_test = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')
sample_submission = pd.read_csv('../input/tabular-playground-series-may-2021/sample_submission.csv') 

# label encode the target column
le = LabelEncoder()
df_train.target = le.fit_transform(df_train.target)
features = df_train.columns[1:51]
#features = [col for col in df_train.columns if col.startswith("feature")]
df_train[features].head()

In [ ]:
# drop duplicates
display(df_train[df_train[features].duplicated(keep="first")])
df_train= df_train.drop_duplicates(subset = features)
display(df_train.shape)

In [ ]:
# let's try to one hot encode all columns that might be "true categoric features". This is just a guess based on my EDA (seperate notebook)
cat_features = ["feature_2", "feature_13", "feature_22", "feature_36"]
#df_train = pd.get_dummies(df_train, columns= cat_features)
#df_test = pd.get_dummies(df_test, columns= cat_features)

In [ ]:
# prepare test data
df_test.head()
X_test=df_test.drop(columns="id")
X_test.head()

In [ ]:
# define X and y
X = df_train.drop(columns=["id","target"])
y = df_train.target
display(X.shape)
df_train.target.value_counts()

In [ ]:
# dividing X, y into train and test data
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state = 0, stratify=y)
display(X_train.shape)
y_train.value_counts()

In [ ]:
# in this cell SMOTE is implemented, can be turned on or off, balanced oversampling is tested and oversampling with a sampling strategy
# I turned if OFF as the results were worse

# scale the data when using SMOTE as it runs a KNN algorithm internally
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_val = scaler.transform(X_val)
#X_test = scaler.transform(X_test)

#oversample = SMOTE() # for balanced oversampling
#strategy = {0:10000, 1:43122, 2:16065, 3:14000} # setting sampling goals manually: only slight increase in underrepresented classes
#oversample = SMOTE(sampling_strategy=strategy) # for strategic oversampling
#X_train, y_train = oversample.fit_resample(X_train, y_train) # oversampling must only be used on training data

#display(X_train.shape)
#y_train.value_counts()

In [ ]:
# train model
clf_xgb = xgb.XGBClassifier(objective='multi:softprob', 
                            seed=42, 
                            use_label_encoder=False, 
                            num_class=4, 
                            colsample_bytree=0.5, 
                            subsample=0.9,
                            eta=0.1,
                            gamma = 0.25,
                            max_depth = 3,
                            reg_lambda = 10,
                            n_estimators = 500
                            )
clf_xgb.fit(X_train, 
            y_train,
            verbose=True,
            ## the next three arguments set up early stopping.
            early_stopping_rounds=50,
            eval_metric=['mlogloss'],
            eval_set=[(X_train, y_train),(X_val, y_val)]
           )
# from Grid Search Round 2: {'gamma': 0.25, 'learning_rate': 0.3, 'max_depth': 3, 'reg_lambda': 10}

In [ ]:
# 1.[58]	validation_0-mlogloss:1.01002	validation_1-mlogloss:1.09621 - with eta= 0.3
# 2.[81]	validation_0-mlogloss:1.01296	validation_1-mlogloss:1.09486 - with eta= 0.2
# 3.[82]	validation_0-mlogloss:1.01188	validation_1-mlogloss:1.09464 - with the 4 duplicates in training data removed
# 4.[76]	validation_0-mlogloss:1.01515	validation_1-mlogloss:1.09554 - with one hot encoding of 4 columns
# 5.[138]	validation_0-mlogloss:1.06422	validation_1-mlogloss:1.09208 - with optimized parameters and increased n_estimators as early stopping was not triggered with 100 estimators
# 6.[159]	validation_0-mlogloss:1.06088	validation_1-mlogloss:1.09217 - one hot encoding removed
# 7.[170]	validation_0-mlogloss:1.05924	validation_1-mlogloss:1.09238 - changed n_estimators from 200 to 500 and early_stopping from 20 to 50
# 8.[499]	validation_0-mlogloss:1.18783	validation_1-mlogloss:1.22255 - with balanced SMOTE, early stopping did not kick in 
# 9.[173]	validation_0-mlogloss:1.08663	validation_1-mlogloss:1.12238 - with strategic SMOTE
# 10.[280]	validation_0-mlogloss:1.05667	validation_1-mlogloss:1.09219 - no SMOTE, eta = 0.2
# 11.[241]	validation_0-mlogloss:1.02529	validation_1-mlogloss:1.09228 - with strategic SMOTE used correctly (scaling and only X_train)
#12.[280]	validation_0-mlogloss:1.05667	validation_1-mlogloss:1.09218 - no SMOTE, just scaling (as expected same as #10.)
#13.[348]	validation_0-mlogloss:1.00963	validation_1-mlogloss:1.09786 - balanced SMOTE used correctly (scaling and only X_train)
#14.[326]	validation_0-mlogloss:1.06012	validation_1-mlogloss:1.09166 - not SMOTE, eta =1.5
#15.[499]	validation_0-mlogloss:1.05940	validation_1-mlogloss:1.09133 - eta =1, used the maximum numbers of estimators

In [ ]:
# 1.Public Score: not submitted
# 2.Public Score: 1.09190
# 3.Public Score: 1.09128
# 4.Public Score: 1.09082
# 5.Public Score: 1.08813
# 6.Public Score: 1.08804
# 7.Public Score: 1.08804
# 8.Public Score: 1.27648
# 9.Public Score: 1.09327
#10.Public Score: 1.08792
#11.Public Score: 1.08950
#12.Public Score: 1.08894
#13.Public Score: 1.09411
#14.Public Score: 1.08779

In [ ]:
# visualize training/validation results
results = clf_xgb.evals_result()

plt.figure(figsize=(10,7))
plt.plot(results["validation_0"]["mlogloss"], label="Training loss")
plt.plot(results["validation_1"]["mlogloss"], label="Validation loss")
plt.axvline(clf_xgb.best_ntree_limit, color="gray", label="Optimal tree number")
plt.annotate(str(clf_xgb.best_ntree_limit), xy=(clf_xgb.best_ntree_limit, 1.2),color="gray")
plt.xlabel("Number of trees")
plt.ylabel("Multiclass LogLoss")
plt.legend()
plt.show()

This plot shows the number of estimators used for the "optimal" result. No need to increase the number of estimators beyond that. 
Depending on whether SMOTE is used or not the picture changes.

In [ ]:
predicted_classes = clf_xgb.predict(X_val) # used only for confusion matrix later, therefore with X_val
predictions = clf_xgb.predict_proba(X_test) # used for submission
predictions

In [ ]:
class_labels = ["Class_1","Class_2","Class_3","Class_4"]
plot_confusion_matrix(clf_xgb, X_val, y_val, display_labels=class_labels)
print("Accuracy: {}\n".format(accuracy_score(y_val,predicted_classes)))
print("Confusion Matrix:")

In [ ]:
pd.Series(predicted_classes).value_counts(normalize=True)
# Class_2 is predicted 98% of the time :(
# when the 4 duplicates in the training set are dropped, Class_1 is not predicted at all :(
# with one hot encoding of 4 columns Class_3 is not predicted at all
# interesting optimization: now it only predicts 2 classes, but logloss is better
# with one hot encoding removed again only Class_1 is not predicted. Accuracy is slightly better
# with balanced SMOTE all classes are predicted, but much more evenly than they should be. !! SMOTE was used wrongly !!
# with strategic SMOTE the confusion matrix looks more similar to no SMOTE. As expected. Now it's Class_1 that is not predicted at all.!! SMOTE was used wrongly !!
# strategic SMOTE used correctly only Class_2 and Class_3 are predicted
# balanced SMOTE used correctly: Class_1 is not predicted

In [ ]:
def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)

plot_features(clf_xgb,(10,12) )
plt.show()

Looking at the feature importance graph, it looks like one hot encoding is quite unimportant. However it led to a small improvement in the Public Score. I will run one last try and remove one hot encoding again. Result: those features are still at the end of the importance list, so one-hot encoding was not the culprit. 

In [ ]:
# create submission file
sample_submission.drop(columns=class_labels, inplace=True)
submission = (sample_submission.join(pd.DataFrame(data=predictions, columns=class_labels)))
submission.to_csv("my_submission.csv", index=False)
submission

In [ ]:
# Grid Search for better hyperparameters
# Round 1
#param_grid = {
#     'max_depth': [3, 4, 5], #4
#     'learning_rate': [0.2, 0.1, 0.05],
#     'gamma': [0, 0.25, 0.5],
#     'reg_lambda': [5.0, 10.0, 15.0] #10
# }
# Round 2
param_grid = {
     'max_depth': [2 ,3, 4],
     'learning_rate': [0.3, 0.2, 0.1],
     'gamma': [0.25],
     'reg_lambda' : [10] #10
 }
optimal_params = GridSearchCV(
     estimator=xgb.XGBClassifier(objective='multi:softprob', eval_metric=['mlogloss'], seed=42, use_label_encoder=False, colsample_bytree=0.5, subsample=0.9),
     param_grid=param_grid,
     scoring = 'neg_log_loss',
     verbose=0, # NOTE: If you want to see what Grid Search is doing, set verbose=2
     cv = 3
 )
# Round 1: {'gamma': 0.25, 'learning_rate': 0.2, 'max_depth': 3, 'reg_lambda': 10.0}
# Round 2: {'gamma': 0.25, 'learning_rate': 0.3, 'max_depth': 3, 'reg_lambda': 10}

In [ ]:
#optimal_params.fit(X_train,y_train,early_stopping_rounds=30,eval_set=[(X_val, y_val)],verbose=50)
#print(optimal_params.best_params_)